In [26]:
import gensim
import pandas as pd
import numpy as np
import csv
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import operator
import gensim.models

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
#load pre-existing model
model = gensim.models.Word2Vec.load("/media/user/Data/wellai/models/word2vec_financial_new2.model")

In [88]:
#file contains mappings from some tickers to some names
df=pd.read_excel('Assets.xlsx')

In [89]:
df.head()

,Ticker,shortName,name,longName
0,NKLA,Nikola,Nikola Motors,Nikola Motors
1,000001 CH Equity,PING AN BANK-A,PING AN BANK CO LTD-A,Ping An Bank Co Ltd
2,000002 CH Equity,CHINA VANKE CO-A,CHINA VANKE CO LTD -A,China Vanke Co Ltd
3,000005 CH Equity,SHENZ FOUNTAIN-A,SHENZHEN FOUNTAIN CORP-A,Shenzhen Fountain Corp
4,000006 CH Equity,SHENZ ZHENYE-A,SHENZHEN ZHENYE GROUP CO-A,Shenzhen Zhenye Group Co Ltd


In [90]:
#Create mapping from ticker to name and from name to ticker to bring order to final prediction dictionary
tickname={}
nametick={}
for index, row in df.iterrows():
    #print(row['Ticker'])
    ticker=row['Ticker'].split()[0].title()
    name=row['name'].split()[0].title()
    tickname[ticker]=name
    nametick[name]=ticker
   

In [97]:
#find most similar stocks & companies
pr=model.wv.most_similar(['Tesla'],topn=200)

In [98]:
finalist={}
#go through model predictions, remove duplicates and create a final dictionary {Name: (Probability, Ticker if avail)}
for p in pr:
    if  len(p[0])>2:
        clean=p[0].replace("’s","")
        clean=clean.replace("'s","").title()
        #print(clean)
        cleanalias=clean
        tickeralias=""
        # a bit of a trick that in text we could see the name or the ticker being mentioned. Properly, it is best to 
        #combine those at the text cleaning stage, so that 'Tesla' and 'TSLA' get put into one entity
        #here we are simply taking the first available in the list with highest probability and  using that
        if clean in tickname:
            cleanalias=tickname[clean]
        if clean in nametick or cleanalias in nametick:
            tickeralias=nametick[cleanalias]
        if cleanalias not in finalist :
            finalist[cleanalias]=(p[1],tickeralias)
        #finalist[entity]=(p[1])
finalist

{'Tesla': (0.814926266670227, 'Tsla'),
 'Musk': (0.7463796138763428, ''),
 'Elon': (0.7269102334976196, ''),
 'Automaker': (0.6607990264892578, ''),
 'Nio': (0.6583075523376465, 'Nio'),
 'Forward': (0.583686351776123, 'Fwrd'),
 'Nikola': (0.5724355578422546, 'Nkla'),
 'Gigafactory': (0.5711787939071655, ''),
 'Cybertruck': (0.5705713629722595, ''),
 'Cadillac': (0.5384987592697144, ''),
 'Rival': (0.5378053784370422, ''),
 'Nvidia': (0.5317240953445435, 'Nvda'),
 'Xpeng': (0.5313283801078796, ''),
 'Cars.Com': (0.5269506573677063, 'Cars'),
 'Sedan': (0.5253123044967651, ''),
 'Porsche': (0.516012966632843, 'Pah3'),
 'Bayerische': (0.5137271881103516, 'Bmwyy'),
 'Toyota': (0.5017447471618652, 'Tyidy'),
 'Crossover': (0.5013091564178467, ''),
 'Suvs': (0.5003219842910767, ''),
 'Maker': (0.49508196115493774, ''),
 'Ebay': (0.49454233050346375, 'Ebay'),
 'Evs': (0.4912528097629547, 'Evs'),
 'F-150': (0.4900296628475189, ''),
 'Sedans': (0.48979905247688293, ''),
 '"Tesla': (0.483615219593

In [86]:
 #correlation between stocks or companies
model.wv.similarity('Tesla','Nio')

0.6583075

In [99]:
#correlation between two concepts
pr2=model.predict_output_word(['Tesla','Nvidia','Recession'], topn=100)


In [96]:
#thematic correlations
pr2=model.predict_output_word(['Biden','Amazon'], topn=100)
final_prob,final=ret_ner(pr2)
final_prob


[('Amazon', 0.7694033),
 ('Apple', 0.15990408),
 ('Warren', 0.0015528984),
 ('Microsoft', 0.0014549826),
 ('House', 0.0013602626),
 ('J.C.', 0.0009885397),
 ('Alphabet', 0.0009004139),
 ('HBO', 0.00038509563),
 ('Trump', 0.00037678654),
 ('Costco', 0.000254567),
 ('Joe', 0.00024465576),
 ('Amazon.com', 0.0002111088),
 ('Warner', 0.00016383898),
 ('Twitter', 0.00011143085),
 ('AT&T', 9.177878e-05),
 ('Harris', 8.600585e-05),
 ('Sunday', 8.020848e-05),
 ('Republican', 6.957982e-05),
 ('giants', 6.0780374e-05),
 ('Senate', 6.0767245e-05),
 ('Zuckerberg', 5.342736e-05),
 ('year', 5.3413456e-05),
 ('eBay', 5.1548126e-05),
 ('Coca-Cola', 5.068186e-05)]